`cd project/seniorthesis/notebooks`

`conda list PyCO2SYS`

`pip install PyCO2SYS`

`pip install git+https://github.com/jbusecke/cmip6_preprocessing.git`

`mv *.png ../figures`

# Transform `lev` dimensions

In [1]:
import xarray as xr
import xesmf as xe
from cmip6_preprocessing.preprocessing import combined_preprocessing
import PyCO2SYS as pyco2 # i have to install this package through terminal for it to work
import intake
import matplotlib.pyplot as plt
import numpy as np
import warnings
import gsw
from cmip6_preprocessing.postprocessing import combine_datasets
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import cmocean
import matplotlib.path as mpath
from cmip6_preprocessing.postprocessing import combine_datasets
from cmip6_preprocessing.postprocessing import merge_variables
from xgcm import Grid

In [2]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")


# `gn` datasets

1. retrieve datasets
2. preprocess
3. merge variables
4. filter for datasets that have all variables 
5. combine datasets + select first ensemble member

In [3]:
variables = ['dissic','talk', 'so', 'thetao', 'fgco2']
z_kwargs = {'consolidated': True, 'use_cftime': True}
query = dict(experiment_id=['historical'], 
             table_id=['Omon'], 
             variable_id=variables,
             grid_label=['gn'],
              source_id=['IPSL-CM6A-LR', 
#                          'MPI-ESM1-2-LR', 
                         'CNRM-ESM2-1', 
                         'CanESM5', 
#                          'MPI-ESM1-2-HR', 
#                          'CMCC-ESM2', 
#                          'GISS-E2-1-G', 
                         'UKESM1-0-LL',
#                          'ACCESS-ESM1-5', 
                         'CanESM5-CanOE' 
#                          'GISS-E2-1-G-CC',
#                          'MPI-ESM-1-2-HAM'
                        ]
            )


cat = col.search(**query)

# print(cat.df['source_id'].unique())
dset_dict_old_gn = cat.to_dataset_dict(zarr_kwargs=z_kwargs, storage_options={'token': 'anon'},
                                preprocess=combined_preprocessing, aggregate=False)

dd_new_gn = merge_variables(dset_dict_old_gn)
# list(dd_new.keys())

temp={}
for name,item in dd_new_gn.items():
    #print(name)
    #print(item.data_vars)
    present = item.data_vars
    if all(i in present for i in variables):
        #print(name)
        temp[name]=item
dd_new_gn2 = temp


def pick_first_member(ds_list, **kwargs):
    return ds_list[0]

dd_new_new_gn = combine_datasets(
    dd_new_gn2,
    pick_first_member,
    match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
)
list(dd_new_new_gn.keys())
# dd_new_new_gn


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.IPSL.IPSL-CM6A-LR.historical.r10i1p1f1.Omon.gn.none.fgco2 failed to combine with :cannot compare cftime.DatetimeGregorian(1850, 1, 16, 12, 0, 0, 0) and cftime.DatetimeProlepticGregorian(1850, 1, 16, 12, 0, 0, 0) (different calendars)
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.MOHC.UKESM1-0-LL.historical.r12i1p1f2.Omon.gn.none.so failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r2i1p1f2.Omon.gn.none.talk failed to combine with :cannot compare cftime.DatetimeGregorian(1850, 1, 16, 12, 0, 0, 0) and cf

['IPSL-CM6A-LR.gn.historical.Omon',
 'CanESM5.gn.historical.Omon',
 'CanESM5-CanOE.gn.historical.Omon',
 'UKESM1-0-LL.gn.historical.Omon',
 'CNRM-ESM2-1.gn.historical.Omon']

## Regridder
I'm using the target grid Julius wrote sort of as a base. I'm subtracting 90 from y so y is -90 to 90 rather than 0 to 180.

There is probably a more efficient way to do this

In [4]:
targetgrid_ds = xe.util.grid_global(1.0, 1.0)
new_y = targetgrid_ds.y-90
# targetgrid_ds = targetgrid_ds.assign_coords({'lev': dd_new_new_gn['CanESM5-CanOE.gn.historical.Omon'].lev})
targetgrid_ds
# # regridder = xe.Regridder(dd_new_new, targetgrid_ds, 'bilinear', 
# #                          periodic=True, ignore_degenerate=True)
# targetgrid_ds
# # yy = targetgrid_ds.y-90
# # targetgrid_ds.assign_coords({'y' : yy})



# target = xr.Dataset({'lat': (['y', 'x'], np.arange(-90, 90, 1.0)),
#                      'lon': (['lon'], np.arange(0, 360, 1.0)),
#                      'y': (['lat'], np.arange(-90, 90, 1.0)),
#                      'x': (['lon'], np.arange(0, 360, 1.0)),
#                     }
#                    )
# target



<xarray.Dataset>
Dimensions:  (x: 360, x_b: 361, y: 180, y_b: 181)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
    lon_b    (y_b, x_b) float64 -180.0 -179.0 -178.0 ... 178.0 179.0 180.0
    lat_b    (y_b, x_b) float64 -90.0 -90.0 -90.0 -90.0 ... 90.0 90.0 90.0 90.0
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    *empty*

In [5]:

dd_regrid={}
for name,item in dd_new_new_gn.items():
    regridder = xe.Regridder(item, targetgrid_ds, 'bilinear', 
                         periodic=True, ignore_degenerate=True)
    ds_regridded = regridder(item)
#     dd_regrid[name]=ds_regridded
    dd_regrid[name] = ds_regridded.assign_coords({'y' : new_y})
list(dd_regrid.keys())

/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:496: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:496: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:496: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:496: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply

['IPSL-CM6A-LR.gn.historical.Omon',
 'CanESM5.gn.historical.Omon',
 'CanESM5-CanOE.gn.historical.Omon',
 'UKESM1-0-LL.gn.historical.Omon',
 'CNRM-ESM2-1.gn.historical.Omon']

In [6]:
variables = ['dissic','talk', 'so', 'thetao', 'fgco2']
z_kwargs = {'consolidated': True, 'use_cftime': True}
query = dict(experiment_id=['historical'], 
             table_id=['Omon'], 
             variable_id=variables,
             grid_label=['gr'],
              source_id=['GFDL-ESM4', 'CESM2-WACCM', 'CESM2', 
                         'CESM2-FV2', 'MRI-ESM2-0', 
                         'GFDL-CM4', 'CESM2-WACCM-FV2']
            )


cat = col.search(**query)

# print(cat.df['source_id'].unique())
dset_dict_old_gr = cat.to_dataset_dict(zarr_kwargs=z_kwargs, storage_options={'token': 'anon'},
                                preprocess=combined_preprocessing, aggregate=False)

dd_new_gr = merge_variables(dset_dict_old_gr)

temp={}
for name,item in dd_new_gr.items():
    #print(name)
    #print(item.data_vars)
    present = item.data_vars
    if all(i in present for i in variables):
        #print(name)
        temp[name]=item
dd_new_gr = temp
# list(dd_new_gr.keys())

dd_new_new_gr = combine_datasets(
    dd_new_gr,
    pick_first_member,
    match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
)
list(dd_new_new_gr.keys())
# dd_new_new_gr


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.NCAR.CESM2-WACCM-FV2.historical.r2i1p1f1.Omon.gr.none.fgco2 failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.NCAR.CESM2-FV2.historical.r2i1p1f1.Omon.gr.none.thetao failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.NCAR.CESM2.historical.r10i1p1f1.Omon.gr.none.so failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/po

['CESM2.gr.historical.Omon',
 'CESM2-FV2.gr.historical.Omon',
 'CESM2-WACCM.gr.historical.Omon',
 'CESM2-WACCM-FV2.gr.historical.Omon',
 'GFDL-ESM4.gr.historical.Omon',
 'MRI-ESM2-0.gr.historical.Omon',
 'GFDL-CM4.gr.historical.Omon']

In [7]:
ds = dd_regrid['IPSL-CM6A-LR.gn.historical.Omon']
grid = Grid(ds, coords={'Z': {'center': 'lev'},
                        },
                periodic=False
            )
grid

<xgcm.Grid>
Z Axis (not periodic, boundary=None):
  * center   lev

In [14]:
ds

<xarray.Dataset>
Dimensions:      (bnds: 2, lev: 75, time: 1980, x: 360, y: 180)
Coordinates:
  * lev          (lev) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    lev_bounds   (lev, bnds) float32 dask.array<chunksize=(75, 2), meta=np.ndarray>
  * time         (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bounds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lon          (y, x) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
    lat          (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
  * y            (y) int64 -90 -89 -88 -87 -86 -85 -84 ... 83 84 85 86 87 88 89
Dimensions without coordinates: bnds, x
Data variables:
    area         (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
    so           (time, lev, y, x) float64 dask.array<chunksize=(5, 75, 180, 360), meta=np.ndarray>
    dissic       (time, lev, y, x) float64 dask.array<chunksize=(5, 75, 180, 360), meta=np.ndarray>
    fgco2        (time, y, x) float64 dask.array<chunksize=(222, 180, 360), meta=np.ndarray>
    talk         (time, lev, y, x) float64 dask.array<chunksize=(5, 75, 180, 360), meta=np.ndarray>
    thetao       (time, lev, y, x) float64 dask.array<chunksize=(4, 75, 180, 360), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [8]:
target_depth_levels=dd_new_new_gr['CESM2-FV2.gr.historical.Omon'].lev

In [12]:
dd_pco2= {}
for name, ds_pco2 in dd_regrid.items():

    if np.max(ds_pco2.lon) > 180:
        dd_pco2[name] = ds_pco2.isel(time = slice(0, 239)).isel(y=slice(0,61)).sel(x=slice(180,200)).mean(['time', 'x'],keep_attrs=True)
    else:
        dd_pco2[name] = ds_pco2.isel(time = slice(0, 239)).isel(y=slice(0,61)).sel(x=slice(0,20)).mean(['time', 'x'],keep_attrs=True)
dd_pco2

{'IPSL-CM6A-LR.gn.historical.Omon': <xarray.Dataset>
 Dimensions:     (bnds: 2, lev: 75, y: 61)
 Coordinates:
   * lev         (lev) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
     lev_bounds  (lev, bnds) float32 dask.array<chunksize=(75, 2), meta=np.ndarray>
   * y           (y) int64 -90 -89 -88 -87 -86 -85 ... -35 -34 -33 -32 -31 -30
 Dimensions without coordinates: bnds
 Data variables:
     area        (y) float64 dask.array<chunksize=(61,), meta=np.ndarray>
     talk        (lev, y) float64 dask.array<chunksize=(75, 61), meta=np.ndarray>
     so          (lev, y) float64 dask.array<chunksize=(75, 61), meta=np.ndarray>
     fgco2       (y) float64 dask.array<chunksize=(61,), meta=np.ndarray>
     thetao      (lev, y) float64 dask.array<chunksize=(75, 61), meta=np.ndarray>
     dissic      (lev, y) float64 dask.array<chunksize=(75, 61), meta=np.ndarray>
 Attributes:
     regrid_method:  bilinear,
 'CanESM5.gn.historical.Omon': <xarray.Dataset>
 Dimensions:     (bnds: 2, lev

In [13]:
grid = Grid(ds, coords={'Z': {'center': 'lev'},
                        },
                periodic=False
            )
target_depth_levels=dd_new_new_gr['CESM2-FV2.gr.historical.Omon'].lev
dd_regrid_new = {}

for name, ds in dd_pco2.items():
    dd_regrid_new[name] = {}
    dd_regrid_new[name]['lev'] = target_depth_levels
    dd_regrid_new[name]['y'] = ds.y
#     dd_regrid_new[name]['x'] = ds.x
    dd_regrid_new[name]['so'] = grid.transform(ds.so, 'Z', target_depth_levels, target_data=None, method='linear')
    dd_regrid_new[name]['thetao'] = grid.transform(ds.thetao, 'Z', target_depth_levels, target_data=None, method='linear')
    dd_regrid_new[name]['talk'] = grid.transform(ds.talk, 'Z', target_depth_levels, target_data=None, method='linear')
    dd_regrid_new[name]['dissic'] = grid.transform(ds.dissic, 'Z', target_depth_levels, target_data=None, method='linear')

In [14]:
dd_regrid_new['IPSL-CM6A-LR.gn.historical.Omon']

{'lev': <xarray.DataArray 'lev' (lev: 33)>
 array([   0.,   10.,   20.,   30.,   50.,   75.,  100.,  125.,  150.,  200.,
         250.,  300.,  400.,  500.,  600.,  700.,  800.,  900., 1000., 1100.,
        1200., 1300., 1400., 1500., 1750., 2000., 2500., 3000., 3500., 4000.,
        4500., 5000., 5500.])
 Coordinates:
   * lev      (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
 Attributes:
     axis:           Z
     bounds:         lev_bnds
     long_name:      ocean model level
     positive:       down
     standard_name:  olevel
     units:          m,
 'y': <xarray.DataArray 'y' (y: 61)>
 array([-90, -89, -88, -87, -86, -85, -84, -83, -82, -81, -80, -79, -78, -77,
        -76, -75, -74, -73, -72, -71, -70, -69, -68, -67, -66, -65, -64, -63,
        -62, -61, -60, -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49,
        -48, -47, -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35,
        -34, -33, -32, -31, -30])
 Coordinates:
   * y        (y) in

In [15]:

def calc_PpCO2_graeme(ds):
    results = pyco2.sys(par1=ds['talk']*1035.0,par2=ds['dissic']*1035.0,par1_type=1,par2_type=2,
                        pressure=0)
    return ds['talk'].copy(data=results['pCO2'])

def calc_PpCO2(ds):
    p = gsw.p_from_z(-1*ds['lev'], ds['y'], geo_strf_dyn_height=0, sea_surface_geopotential=0)
    insitutemp = gsw.t_from_CT(ds['so'], ds['thetao'], p)  # 0 is just a filler rn 
    conversion =  1e6/1035
    results = pyco2.sys(par1=ds['talk']*conversion,par2=ds['dissic']*conversion,par1_type=1,par2_type=2,
                        pressure_out=0, temperature_out = ds['thetao'], pressure = p, 
                        temperature = insitutemp)
    return ds['talk'].copy(data=results['pCO2_out'])

def meridionalsection(ax,da,clims=None,title=None, label = None):
    im = ax.pcolormesh(da['y'],da['lev'],da)
    if clims is not None:
        im.set_clim(clims)
    ax.set_xlim([-80,60])
    ax.invert_yaxis()
    cbar = plt.colorbar(im,ax=ax)
    cbar.set_label(label)
    ax.set_title(title)
    
def calc_sigma2(ds):
    return gsw.sigma2(ds['so'],ds['thetao'])

def meridionalsection_with_sigma2(ax,da,sigma2,clims=None,title=None, label = None):
    meridionalsection(ax,da,clims, label = label)
    ax.contour(da['y'],da['lev'],sigma2,levels=[36,36.4,36.8],colors='w')
    ax.set_title(title)
    
def draw_SOcustom(X,Y,Z,cmap,cbar_label,title,plot_type,**kwargs):
    
    crs_plot = ccrs.SouthPolarStereo()
    crs_source = ccrs.PlateCarree()
    
    fig,ax = plt.subplots(figsize=(10,10),subplot_kw={'projection':crs_plot})
    ax.add_feature(cfeature.LAND,zorder=10,facecolor='darkgray')
    ax.set_extent([-180, 180, -90, -30], crs_source)
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    
    ax.gridlines()
    ax.coastlines()
    
    ax.set_title(title,fontsize=14,pad = 20)
    
    if plot_type == 'contourf':
        im = ax.contourf(X,Y,Z,kwargs['clevs'],cmap=cmap,transform=crs_source)
    elif plot_type == 'pcolormesh':
        im = ax.pcolormesh(X,Y,Z,cmap=cmap,transform=crs_source,vmin=kwargs['clims'][0],vmax=kwargs['clims'][1])
    
    cbar = plt.colorbar(im,orientation='horizontal',fraction=0.025,pad=0.05)
    cbar.set_label(cbar_label,fontsize=12)
    
#     plt.show()
    
    return fig,ax

In [64]:
dd_regrid_new['IPSL-CM6A-LR.gn.historical.Omon']

{'lev': <xarray.DataArray 'lev' (lev: 33)>
 array([   0.,   10.,   20.,   30.,   50.,   75.,  100.,  125.,  150.,  200.,
         250.,  300.,  400.,  500.,  600.,  700.,  800.,  900., 1000., 1100.,
        1200., 1300., 1400., 1500., 1750., 2000., 2500., 3000., 3500., 4000.,
        4500., 5000., 5500.])
 Coordinates:
   * lev      (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
 Attributes:
     axis:           Z
     bounds:         lev_bnds
     long_name:      ocean model level
     positive:       down
     standard_name:  olevel
     units:          m,
 'y': <xarray.DataArray 'y' (y: 180)>
 array([-90, -89, -88, -87, -86, -85, -84, -83, -82, -81, -80, -79, -78, -77,
        -76, -75, -74, -73, -72, -71, -70, -69, -68, -67, -66, -65, -64, -63,
        -62, -61, -60, -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49,
        -48, -47, -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35,
        -34, -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23,

In [ ]:
calc_PpCO2(dd_regrid_new['IPSL-CM6A-LR.gn.historical.Omon'])